<a href="https://colab.research.google.com/github/asigalov61/AlexMIDIDataSet/blob/master/Simple_MIDI_Reducer_Parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple MIDI Reducer/Parser

### Created by Ravi A. and all credit goes out to him for his amazing and simple reducer code :)

## Setup Environment and Dependencies

In [ ]:
#@title Start Here to setup the Environment and Dependencies
!pip install pretty_midi
import pretty_midi
import glob
import os
import random
import IPython.display as ipd

### Create IO directories if do not exist

In [ ]:
#@title Choose names for Source and Destination folders for conversion
source_dir = "alex_midi_original" #@param {type:"string"}
dest_dir = "alex_midi_piano_only" #@param {type:"string"}
source_path = source_dir
dest_path = dest_dir
if not os.path.exists(source_path):
    os.makedirs(source_path)
if not os.path.exists(dest_path):
    os.makedirs(dest_path)

Download and Unzip Alex MIDI Dataset (Multi-Instrumental with Drums) to the source directory specified in the previous section

In [ ]:
#@title Download and Unzip Alex Piano MIDI Dataset (~400 MIDIs)
%cd /content/alex_midi_original/
!wget 'https://github.com/asigalov61/AlexMIDIDataSet/raw/master/AlexMIDIDataSet-CC-BY-NC-SA.zip'
!unzip -j 'AlexMIDIDataSet-CC-BY-NC-SA.zip'
%cd /content/

In [ ]:
#@title Create a list of paths and files. Select desired MIDI extension
files_extension = "*.mid" #@param ["*.mid", "*.midi"]
source_files = glob.glob(os.path.join(source_path, files_extension))

In [ ]:
#@title Define reducer/parser function and input desired instrument to reduce to
is_it_drums = False #@param {type:"boolean"}
desired_instrument_to_reduce_to = "piano" #@param {type:"string"}
def filter_piano(fname):
    pm = pretty_midi.PrettyMIDI(fname)
    outmid = pretty_midi.PrettyMIDI()
    inst = pretty_midi.Instrument(program=0, is_drum=is_it_drums, name=desired_instrument_to_reduce_to)
    outmid.instruments.append(inst)
    added=0
    for instrument in pm.instruments:
        if instrument.program ==0 and not instrument.is_drum:
            print('Adding', instrument)
            added+=1
            for note in instrument.notes:
                inst.notes.append(note)
        else:
            print('Skipping', instrument)
    
    return pm, outmid, added

### Reducer-Parser-Filter Loop/Main Function

In [ ]:
#@title Run this to reduce/filter all files. Select output files extension and name tag
output_extension = ".mid" #@param {type:"string"}
output_files_names_tag = "_piano.mid" #@param {type:"string"}
#fname = random.choice(source_files)
for fname in source_files:
    print("Reading:", fname)
    pm, outmid, added =filter_piano(fname)
    if added>0:
        outname = fname.replace(output_extension, output_files_names_tag).replace(source_path, dest_path)
        print("Writing", outname)
        outmid.write(outname)
        #audio = pm.synthesize()
        #audio2 = outmid.synthesize()
    print()

In [ ]:
#@title Synth last converted MIDI and play it as WAV (Yes, I know this synthesizer sucks. Help to change it to Fluidsynth at least. Thanx.)
audio2 = outmid.synthesize()
ipd.Audio(audio2, rate=44100)